In [1]:
import csv
import os
import sys
import os.path
from gensim.models import Word2Vec
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from gensim.scripts.glove2word2vec import glove2word2vec


In [2]:
#create dictionary of words in vocab
vocab = {}
with open('glove.6B.100d.txt' , 'r') as inpfile:
    line = inpfile.readline()
    while line:
        line = line.strip()
        line = line.split()
        vocab[line[0]] = 1
        line = inpfile.readline()
print('Done making vocab')

#load word2vec embedding

glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
print ('Done loading embedding')

Done making vocab
Done loading embedding


In [3]:
#function returns cosine similartiy between given words
def calculate_cosine_similartiy(word1 , word2):
    A = model[word1 ]
    A = np.reshape(A, (1, -1))
    B = model[word2 ]
    B = np.reshape(B, (1, -1))
    x = cosine_similarity( A , B )[0][0]
    return x
    

In [4]:
#convert training data in representational format

cosine_similarity_words_features = []  
concatinated_word_embeddings     = []
label = []


total = 0   #total rows in train data
cnt   = 0   #points present in vocab
output_file_one  = open('train_output_one.txt', 'w')
output_file_zero = open('train_output_zero.txt', 'w')


with open('train.txt') as train_file:
    reader = csv.reader(train_file)
    for row in reader :
        total += 1
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            cnt += 1
            continue
        cosine_similarity_between_w1_atb = calculate_cosine_similartiy(row[0] , row[2])
        cosine_similarity_between_w2_atb = calculate_cosine_similartiy(row[1] , row[2])
        #print(cosine_similarity_between_w1_atb , cosine_similarity_between_w2_atb)
        if (row[3]=='1'):
            output_file_one.write(row[0]+" " + row[1]+" " + row[2] +'\n' )
            output_file_one.write(str(cosine_similarity_between_w1_atb)+" " + str(cosine_similarity_between_w2_atb)+" " + str( row[3] )+'\n' )
        else:
            output_file_zero.write(row[0]+" " + row[1]+" " + row[2] +'\n' )
            output_file_zero.write(str(cosine_similarity_between_w1_atb)+" " + str(cosine_similarity_between_w2_atb)+" " + str( row[3] )+'\n' )
        
        label.append([int(row[3])])
        cosine_similarity_words_features.append([cosine_similarity_between_w1_atb , cosine_similarity_between_w2_atb ])
        concatinated_list = (model[row[0]]  +  model[row[1]]  + model[row[2]] ).tolist()
        concatinated_word_embeddings.append(concatinated_list  )
        
print('Done representing')
    


Done representing


In [5]:
#train logistic regression model by using cosine values as feature vector

logistic_regr_model_cosine_sim   = LogisticRegression(max_iter = 300)
logistic_regr_model_cosine_sim.fit( cosine_similarity_words_features , label )

print("Done fitting")

Done fitting


In [6]:
#train logistic regression model by concatinating word embedding values as feature vector


logistic_regr_model_concatinated = LogisticRegression(max_iter = 300)
logistic_regr_model_concatinated.fit( concatinated_word_embeddings , label  ) 
print("Done fitting")


Done fitting


In [17]:
#validate using logistic regression model using cosine values as features vector

total =0
cnt = 0
true_positive  = 0
true_negative  = 0
false_positive = 0
false_negative = 0
with open('validation.txt') as validation_file:
    reader = csv.reader(validation_file)
    for row in reader :
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            continue
        total += 1
        
        cosine_similarity_between_w1_atb = calculate_cosine_similartiy(row[0] , row[2])
        cosine_similarity_between_w2_atb = calculate_cosine_similartiy(row[1] , row[2])
        
        predicted_class = logistic_regr_model_cosine_sim.predict( [cosine_similarity_between_w1_atb , cosine_similarity_between_w2_atb] )
        
        #print(predicted_class , row[3])
        
        if(predicted_class[0] == int(row[3])):
            cnt += 1
            
        if(int(row[3]) == 1):
            if(predicted_class[0]==1):
                true_positive += 1
            else:
                false_negative += 1
        else:
            if(predicted_class[0]==1):
                false_positive += 1
            else:
                true_negative += 1
        
precision = true_positive/( true_positive + false_positive)
recall    = true_positive/( true_positive + false_negative)
            
print (cnt , total , cnt/total)
print ('precision = ' , precision , ' recall = ' , recall )
print ( 'F1 score = ' , 2*((precision*recall)/ (precision+recall))  )
print('done validating')

1341 2721 0.4928335170893054
precision =  0.49402670414617006  recall =  0.5157740278796772
F1 score =  0.5046661880832735
done validating


In [18]:
#validate using logistic regression model using concatinating embedding as features vector

total =0
cnt = 0
true_positive  = 0
true_negative  = 0
false_positive = 0
false_negative = 0

with open('validation.txt') as validation_file:
    reader = csv.reader(validation_file)
    for row in reader :
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            continue
        total += 1
        concatinated_list = (model[row[0]]  +  model[row[1]]  + model[row[2]] ).tolist()
        predicted_class = logistic_regr_model_concatinated.predict(concatinated_list)
        
        #print( predicted_class , row[3])
        if(predicted_class[0]==int(row[3])):
            cnt += 1
        if(int(row[3]) == 1):
            if(predicted_class[0]==1):
                true_positive += 1
            else:
                false_negative += 1
        else:
            if(predicted_class[0]==1):
                false_positive += 1
            else:
                true_negative += 1
        
precision = true_positive/( true_positive + false_positive)
recall    = true_positive/( true_positive + false_negative)
            
print (cnt , total , cnt/total)
print ('precision = ' , precision , ' recall = ' , recall )
print ( 'F1 score = ' , 2*((precision*recall)/ (precision+recall))  )
print('done validating')

1354 2721 0.4976111723631018
precision =  0.4978021978021978  recall =  0.33235509904622157
F1 score =  0.3985921689397272
done validating


In [11]:
#Gaussian Naive bayes using cosine_values as embeddings
from sklearn.naive_bayes import GaussianNB 

gnb_cosine = GaussianNB()
NB_model_cosine = gnb_cosine.fit(cosine_similarity_words_features , label)
gnb_concat = GaussianNB()
NB_model_concat = gnb_concat.fit(concatinated_word_embeddings , label)

print("Done model fitting")

Done model fitting


In [19]:
#validate using Naive Baise model using cosine values as features vector

total =0
cnt = 0
true_positive  = 0
true_negative  = 0
false_positive = 0
false_negative = 0
with open('validation.txt') as validation_file:
    reader = csv.reader(validation_file)
    for row in reader :
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            continue
        total += 1
        
        cosine_similarity_between_w1_atb = calculate_cosine_similartiy(row[0] , row[2])
        cosine_similarity_between_w2_atb = calculate_cosine_similartiy(row[1] , row[2])
        
        predicted_class = NB_model_cosine.predict( [cosine_similarity_between_w1_atb , cosine_similarity_between_w2_atb] )
        
        #print(predicted_class , row[3])
        
        if(predicted_class[0] == int(row[3])):
            cnt += 1
        if(int(row[3]) == 1):
            if(predicted_class[0]==1):
                true_positive += 1
            else:
                false_negative += 1
        else:
            if(predicted_class[0]==1):
                false_positive += 1
            else:
                true_negative += 1
        
precision = true_positive/( true_positive + false_positive)
recall    = true_positive/( true_positive + false_negative)
            
print (cnt , total , cnt/total)
print ('precision = ', precision , ' recall = ' , recall )
print ( 'F1 score = ' , 2*((precision*recall)/ (precision+recall))  )
print('done validating')

1242 2721 0.4564498346196251
precision =  0.46432964329643295  recall =  0.5539251650770359
F1 score =  0.5051856808297089
done validating


In [20]:
#validate using Naive Bayes model using concatinating embedding as features vector

total =0
cnt = 0
total =0
cnt = 0
true_positive  = 0
true_negative  = 0
false_positive = 0
false_negative = 0
with open('validation.txt') as validation_file:
    reader = csv.reader(validation_file)
    for row in reader :
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            continue
        total += 1
        concatinated_list = (model[row[0]]  +  model[row[1]]  + model[row[2]] ).tolist()
        predicted_class = NB_model_concat.predict(concatinated_list)
        
        #print( predicted_class , row[3])
        if(predicted_class[0]==int(row[3])):
            cnt += 1
        
        if(int(row[3]) == 1):
            if(predicted_class[0]==1):
                true_positive += 1
            else:
                false_negative += 1
        else:
            if(predicted_class[0]==1):
                false_positive += 1
            else:
                true_negative += 1
        
precision = true_positive/( true_positive + false_positive)
recall    = true_positive/( true_positive + false_negative)
            
print (cnt , total , cnt/total)
print ('precision = ',  precision , ' recall = ' ,recall )
print ( 'F1 score = ' , 2*((precision*recall)/ (precision+recall))  )
print('done validating')

1223 2721 0.4494671076809996
precision =  0.4432296047098402  recall =  0.38664710198092445
F1 score =  0.41300940438871475
done validating


In [27]:
#calculating cosine similarity using wordnet features 

#convert training data in representational format
from nltk.corpus import wordnet


cosine_similarity_words_features = []  
concatinated_word_embeddings     = []
label = []


total = 0   #total rows in train data
cnt   = 0   #points present in vocab
min_syn_in_any_word = 50

with open('train.txt') as train_file:
    reader = csv.reader(train_file)
    for row in reader :
        total += 1
        if (row[0] not in vocab) or (row[1] not in vocab) or (row[2] not in vocab):
            cnt += 1
            continue
        
        synonyms_word1 = []
        synonyms_word2 = []
        for syn in wordnet.synsets(row[0]):
            for l in syn.lemmas():
                synonyms_word1.append(l.name())
        for syn in wordnet.synsets(row[1]):
            for l in syn.lemmas():
                synonyms_word2.append(l.name())
        synonyms_word1 = set(synonyms_word1)
        synonyms_word2 = set(synonyms_word2)
        
        if(len(synonyms_word1)<5 ):
            print(row[0] , len(synonyms_word1))
        if(len(synonyms_word2)<5 ):
            print(row[1] , len(synonyms_word2))
        
        print( len(synonyms_word1) , len(synonyms_word2) )
        label.append([int(row[3])])
        #cosine_similarity_words_features.append([cosine_similarity_between_w1_atb , cosine_similarity_between_w2_atb ])
        #concatinated_list = (model[row[0]]  +  model[row[1]]  + model[row[2]] ).tolist()
        #concatinated_word_embeddings.append(concatinated_list  )
print (min_syn_in_any_word)
print('Done representing')





9 14
banjo 1
harmonica 4
1 4
mushroom 3
onions 3
3 3
cucumber 4
20 4
54 8
14 12
cedar 4
4 5
sandals 1
1 5
cupboard 2
7 2
olive 3
onions 3
3 3
7 8
stork 1
5 1
clamp 2
hose 4
2 4
11 10
nectarine 3
pineapple 4
3 4
8 7
6 16
34 8
9 13
asparagus 3
5 3
pear 3
3 5
missile 2
8 2
umbrella 1
trousers 2
1 2
pheasant 1
deer 2
1 2
woodpecker 3
9 3
muzzle 3
3 10
eel 1
1 16
scarf 2
5 2
13 21
13 13
mackerel 1
7 1
hatchet 2
sledgehammer 3
2 3
saxophone 2
5 2
giraffe 3
hyena 2
3 2
pear 3
cranberry 1
3 1
clarinet 1
cello 2
1 2
moose 4
13 4
34 8
rattlesnake 2
6 2
skis 1
skateboard 1
1 1
onions 3
3 5
tray 1
1 12
escalator 4
elevator 2
4 2
tray 1
1 9
alligator 2
moose 4
2 4
6 8
accordion 3
3 6
blouse 1
5 1
tricycle 3
3 24
asparagus 3
onions 3
3 3
46 6
5 37
sailboat 2
yacht 2
2 2
tractor 1
1 5
5 7
9 6
7 20
9 5
tractor 1
11 1
tractor 1
11 1
moose 4
4 8
spinach 4
pumpkin 4
4 4
mackerel 1
trout 1
1 1
guitar 1
violin 2
1 2
snail 2
cockroach 2
2 2
plum 4
strawberry 3
4 3
cucumber 4
garlic 3
4 3
10 15
10 10
falcon 

pheasant 1
6 1
carpet 3
5 3
8 10
5 9
12 5
zucchini 2
6 2
5 41
razor 1
axe 2
1 2
lobster 1
5 1
crocodile 1
leopard 2
1 2
10 12
6 5
door 4
8 4
dolphin 3
tiger 2
3 2
6 33
16 5
6 13
9 19
canoe 1
boat 4
1 4
fridge 2
8 2
carpet 3
3 5
7 14
corkscrew 3
3 5
eel 1
5 1
sledgehammer 3
chisel 3
3 3
14 20
tomato 4
7 4
cheetah 3
3 10
grapefruit 2
tomato 4
2 4
trombone 1
harpsichord 2
1 2
hoe 1
1 6
22 21
crocodile 1
cheetah 3
1 3
cow 3
5 3
bagpipe 1
trombone 1
1 1
mirror 1
lamp 1
1 1
cranberry 1
6 1
caterpillar 3
jeep 2
3 2
raccoon 2
coyote 4
2 4
7 38
otter 1
mink 2
1 2
7 6
crocodile 1
alligator 2
1 2
jeans 4
4 39
6 39
46 15
squid 3
27 3
onions 3
14 3
yacht 2
pier 4
2 4
envelope 2
2 16
6 5
9 13
peas 2
6 2
pear 3
onions 3
3 3
helicopter 4
missile 2
4 2
bagpipe 1
violin 2
1 2
7 38
salamander 4
6 4
oriole 4
4 5
sofa 3
3 7
asparagus 3
14 3
trousers 2
2 5
oriole 4
8 4
6 6
motorcycle 4
4 7
wheelbarrow 4
4 24
mackerel 1
trout 1
1 1
shirt 1
1 8
8 19
porcupine 2
2 5
camel 1
calf 3
1 3
ashtray 1
ladle 3
1 3
cal

6 3
beetle 4
spider 2
4 2
hornet 1
9 1
knife 3
41 3
9 8
sailboat 2
yacht 2
2 2
10 21
chipmunk 1
otter 1
1 1
oriole 4
4 8
eel 1
1 16
scarf 2
5 2
olive 3
3 5
pear 3
11 3
pelican 1
penguin 1
1 1
tomato 4
4 7
onions 3
5 3
pistol 4
crossbow 1
4 1
10 7
seaweed 1
1 33
tuba 3
harpsichord 2
3 2
cauliflower 2
7 2
8 6
groundhog 3
raccoon 2
3 2
camel 1
giraffe 3
1 3
spoon 3
onions 3
3 3
cucumber 4
10 4
pigeon 1
30 1
doorknob 2
ashtray 1
2 1
7 6
mink 2
otter 1
2 1
otter 1
1 28
cockroach 2
beetle 4
2 4
pineapple 4
4 20
helmet 1
1 14
54 21
faucet 2
2 19
parsley 2
6 2
grapefruit 2
5 2
spinach 4
21 4
jeans 4
trousers 2
4 2
coyote 4
10 4
7 24
fridge 2
5 2
22 8
door 4
37 4
screwdriver 1
1 11
harpsichord 2
cello 2
2 2
pelican 1
1 14
7 6
rocker 4
guitar 1
4 1
19 5
5 8
8 9
shotgun 2
grenade 1
2 1
8 7
mittens 1
scarf 2
1 2
6 20
scarf 2
8 2
mittens 1
1 9
6 30
tuba 3
accordion 3
3 3
elephant 1
cow 3
1 3
mackerel 1
1 5
cigar 1
5 1
corkscrew 3
3 5
shirt 1
5 1
pheasant 1
11 1
harpoon 1
tomahawk 2
1 2
sofa 3
3 7
c

church 4
2 4
10 6
hatchet 2
axe 2
2 2
honeydew 2
nectarine 3
2 3
moose 4
4 9
walnut 2
oak 2
2 2
garlic 3
21 3
6 13
asparagus 3
27 3
12 7
bra 3
shirt 1
3 1
goldfish 2
trout 1
2 1
rattlesnake 2
2 13
rhubarb 3
11 3
lantern 1
chandelier 3
1 3
camisole 2
2 7
5 5
chickadee 1
owl 4
1 4
8 16
5 5
hatchet 2
sledgehammer 3
2 3
vulture 4
falcon 1
4 1
deer 2
sheep 1
2 1
unicycle 2
skateboard 1
2 1
pheasant 1
asparagus 3
1 3
seagull 3
pheasant 1
3 1
lobster 1
5 1
19 7
trumpet 4
harpsichord 2
4 2
elephant 1
30 1
leopard 2
alligator 2
2 2
5 5
tomato 4
9 4
motorcycle 4
11 4
machete 3
17 3
shawl 1
5 1
5 13
apple 3
3 11
thermometer 1
screwdriver 1
1 1
elephant 1
hyena 2
1 2
pheasant 1
trout 1
1 1
tractor 1
1 5
tangerine 2
2 9
pigeon 1
pheasant 1
1 1
clamp 2
2 15
9 13
5 9
trombone 1
1 6
rattlesnake 2
2 13
bayonet 1
1 15
squirrel 1
5 1
dishwasher 3
7 3
41 11
10 39
13 6
saxophone 2
2 6
cottage 2
11 2
dishwasher 3
3 16
celery 3
5 3
celery 3
asparagus 3
3 3
hamster 1
8 1
barn 2
9 2
grapefruit 2
tomato 4
2 4
p

octopus 2
squid 3
2 3
37 5
onions 3
carrot 3
3 3
mittens 1
trousers 2
1 2
screwdriver 1
1 11
12 12
rooster 2
2 8
knife 3
spatula 1
3 1
motorcycle 4
9 4
6 14
camel 1
deer 2
1 2
canoe 1
boat 4
1 4
6 9
raccoon 2
mink 2
2 2
10 25
trout 1
squid 3
1 3
38 8
5 5
missile 2
tomahawk 2
2 2
pelican 1
1 8
falcon 1
1 9
leopard 2
2 34
tomato 4
10 4
balloon 3
3 7
harmonica 4
guitar 1
4 1
woodpecker 3
owl 4
3 4
vulture 4
eagle 3
4 3
6 8
20 6
21 5
blouse 1
7 1
otter 1
5 1
rattlesnake 2
squirrel 1
2 1
pigeon 1
1 8
blender 3
3 8
hornet 1
falcon 1
1 1
16 10
raccoon 2
2 13
cauliflower 2
onions 3
2 3
finch 1
5 1
penguin 1
pelican 1
1 1
cabin 1
cupboard 2
1 2
blueberry 2
11 2
caribou 4
7 4
grenade 1
bazooka 1
1 1
14 6
cheetah 3
3 7
ox 2
2 6
walnut 2
garlic 3
2 3
accordion 3
3 5
10 46
nectarine 3
grapefruit 2
3 2
garlic 3
33 3
6 5
shirt 1
5 1
8 5
cranberry 1
apple 3
1 3
9 5
jeep 2
7 2
hatchet 2
6 2
6 21
cucumber 4
20 4
5 10
turnip 3
7 3
leopard 2
7 2
seagull 3
3 7
shirt 1
5 1
coyote 4
4 8
tangerine 2
pear 3
2 

...

raisin 1
2 1
walrus 3
hyena 2
3 2
beets 4
onions 3
4 3
dagger 3
3 6
18 15
honeydew 2
2 5
garlic 3
parsley 2
3 2
spinach 4
parsley 2
4 2
7 5
5 6
14 11
mushroom 3
carrot 3
3 3
27 8
plum 4
4 25
beets 4
cauliflower 2
4 2
pigeon 1
1 6
39 6
sheep 1
1 5
7 21
cheetah 3
otter 1
3 1
hare 2
mink 2
2 2
7 9
saxophone 2
clarinet 1
2 1
6 6
pie 3
mushroom 3
3 3
peas 2
2 10
broccoli 2
turnip 3
2 3
12 19
carrot 3
8 3
porcupine 2
deer 2
2 2
5 8
5 5
machete 3
17 3
11 5
sword 4
5 4
11 14
moose 4
4 9
9 8
bottle 4
6 4
7 8
knife 3
dagger 3
3 3
eagle 3
woodpecker 3
3 3
cranberry 1
raisin 1
1 1
30 8
crayon 2
2 6
crayon 2
2 6
chapel 2
2 7
banjo 1
guitar 1
1 1
balloon 3
boat 4
3 4
sword 4
armour 2
4 2
deer 2
2 5
wheelbarrow 4
sled 3
4 3
bayonet 1
1 10
shotgun 2
2 7
tiger 2
2 16
truck 3
5 3
squirrel 1
otter 1
1 1
spoon 3
10 3
toaster 2
freezer 4
2 4
necklace 1
emerald 1
1 1
scarf 2
5 2
slippers 4
mittens 1
4 1
17 21
21 5
honeydew 2
grapefruit 2
2 2
apron 3
3 5
6 7
7 16
24 46
camel 1
1 9
bottle 4
4 9
trousers 2
8 2

kite 1
starling 1
1 1
cow 3
bison 1
3 1
axe 2
6 2
camel 1
1 9
cockroach 2
hyena 2
2 2
plum 4
blueberry 2
4 2
sweater 3
15 3
7 5
trout 1
1 16
coyote 4
4 13
5 5
pencil 1
41 1
airplane 3
21 3
harmonica 4
clarinet 1
4 1
beets 4
celery 3
4 3
tortoise 1
leopard 2
1 2
goldfish 2
2 7
olive 3
parsley 2
3 2
5 6
8 9
5 6
pelican 1
1 14
tomato 4
spinach 4
4 4
ashtray 1
chandelier 3
1 3
grenade 1
1 19
basement 2
cellar 4
2 4
onions 3
asparagus 3
3 3
blouse 1
shirt 1
1 1
mackerel 1
27 1
pistol 4
4 19
corkscrew 3
3 8
10 39
22 10
walrus 3
whale 4
3 4
16 5
scarf 2
5 2
boat 4
yacht 2
4 2
10 6
machete 3
6 3
celery 3
7 3
shawl 1
39 1
sweater 3
6 3
nectarine 3
11 3
pigeon 1
11 1
apron 3
3 8
garlic 3
7 3
asparagus 3
zucchini 2
3 2
revolver 4
10 4
otter 1
5 1
cellar 4
7 4
10 7
7 7
porcupine 2
caribou 4
2 4
owl 4
8 4
corkscrew 3
3 8
owl 4
7 4
shirt 1
blouse 1
1 1
balloon 3
3 21
rhubarb 3
garlic 3
3 3
8 18
moose 4
7 4
tractor 1
6 1
5 30
strawberry 3
3 10
34 38
lobster 1
1 5
12 10
tray 1
7 1
nectarine 3
plum 4
3

beetle 4
2 4
5 5
fridge 2
5 2
28 20
pumpkin 4
14 4
pumpkin 4
8 4
tiger 2
elephant 1
2 1
shirt 1
9 1
14 33
harmonica 4
accordion 3
4 3
iguana 3
crocodile 1
3 1
guitar 1
saxophone 2
1 2
snail 2
2 5
helicopter 4
4 21
tangerine 2
2 10
door 4
7 4
5 5
blouse 1
5 1
apple 3
3 33
mixer 3
blender 3
3 3
beetle 4
6 4
iguana 3
5 3
11 21
balloon 3
27 3
12 16
26 17
razor 1
dagger 3
1 3
33 7
bookcase 1
cupboard 2
1 2
truck 3
8 3
seagull 3
3 7
zebra 1
5 1
boat 4
airplane 3
4 3
cup 4
22 4
5 8
7 5
shotgun 2
pistol 4
2 4
toaster 2
freezer 4
2 4
6 21
tractor 1
11 1
trousers 2
6 2
41 10
sweater 3
5 3
trout 1
5 1
mittens 1
trousers 2
1 2
scarf 2
2 9
squid 3
3 11
spider 2
38 2
9 5
tortoise 1
7 1
truck 3
motorcycle 4
3 4
zucchini 2
5 2
trout 1
squid 3
1 3
grapefruit 2
pineapple 4
2 4
apron 3
6 3
snail 2
2 5
razor 1
1 11
toaster 2
oven 1
2 1
apron 3
mittens 1
3 1
banjo 1
1 6
16 9
tongs 2
ladle 3
2 3
16 5
sleigh 3
sled 3
3 3
violin 2
keyboard 1
2 1
raisin 1
cranberry 1
1 1
11 14
trousers 2
12 2
tortoise 1
gorill

8 5
shawl 1
shirt 1
1 1
5 5
tray 1
7 1
coin 3
3 24
13 39
tricycle 3
unicycle 2
3 2
cucumber 4
19 4
tuba 3
banjo 1
3 1
mackerel 1
1 5
33 6
broccoli 2
2 8
sword 4
dagger 3
4 3
5 21
door 4
brick 1
4 1
basement 2
cellar 4
2 4
lobster 1
1 16
octopus 2
2 5
vulture 4
woodpecker 3
4 3
parsley 2
19 2
cupboard 2
16 2
eagle 3
kite 1
3 1
bathtub 4
4 16
shirt 1
21 1
thermometer 1
lamp 1
1 1
banana 2
11 2
7 5
5 7
escalator 4
elevator 2
4 2
skillet 3
3 54
rocker 4
4 7
door 4
4 5
crocodile 1
tiger 2
1 2
flea 1
beetle 4
1 4
strawberry 3
pineapple 4
3 4
ant 3
3 30
cedar 4
4 9
21 7
7 16
sheep 1
38 1
shirt 1
1 39
slippers 4
8 4
5 9
bookcase 1
12 1
shirt 1
12 1
7 5
fridge 2
8 2
spoon 3
3 21
balloon 3
helicopter 4
3 4
cedar 4
4 7
apple 3
celery 3
3 3
ladle 3
6 3
starling 1
11 1
carpet 3
sofa 3
3 3
rattlesnake 2
iguana 3
2 3
giraffe 3
leopard 2
3 2
7 33
elevator 2
escalator 4
2 4
5 11
typewriter 1
pencil 1
1 1
fridge 2
freezer 4
2 4
gorilla 2
squirrel 1
2 1
bottle 4
bucket 3
4 3
trousers 2
39 2
alligator 2
o

9 18
bra 3
jeans 4
3 4
blouse 1
trousers 2
1 2
trousers 2
5 2
crayon 2
2 6
elevator 2
escalator 4
2 4
cucumber 4
6 4
tiger 2
elephant 1
2 1
22 21
ashtray 1
bucket 3
1 3
scarf 2
5 2
leopard 2
gorilla 2
2 2
16 27
gorilla 2
camel 1
2 1
19 5
apple 3
celery 3
3 3
willow 2
vine 1
2 1
cucumber 4
4 7
10 5
cottage 2
2 5
lantern 1
chandelier 3
1 3
camisole 2
2 39
5 7
truck 3
27 3
garlic 3
5 3
50
Done representing
